## Instrumenting Libraries with Python

The [OpenTelemetry](https://opentelemetry.io/) API lets library authors instrument their code in a vendor-neutral way.

If a developer uses an OpenTelemetry-aware library, they're able to send metrics, traces and events generated by that library to the backend service(s) of their choice without any extra code.

This example uses the OpenTelemetry API to instrument a method that orders donuts. Later, Lightstep's OpenTelemetry launcher automatically generates a configuration to send traces created using OpenTelemetry to Lightstep. Data can also be sent to Prometheus, Datadog, AWS X-Ray, Azure Monitor, New Relic, Google Cloud (and more) without any code changes needed.

There is a growing number of libraries and plugins that already exist for multiple languages and sevices. A (mostly) complete listing is available on the [OpenTelemetry Registry](https://opentelemetry.io/registry/).

In [ ]:
! pip install opentelemetry-api==0.17b0

#### Creating a trace

Here's an example of what a library author would use to either create traces or inspect traces. The only dependency should be the OpenTelemetry API module (not the SDK).

A technical introduction to tracing that introduces many of these concepts—spans, attributes, and events—is [available on Lightstep.](https://opentelemetry.lightstep.com/)


In [ ]:
from opentelemetry import trace
import pprint

pp = pprint.PrettyPrinter(indent=4)

# API docs: https://opentelemetry-python.readthedocs.io/en/stable/api/api.html

def get_donuts():
  tracer = trace.get_tracer('DonutsLibrary', '0.0.1')
  # Creates a new span called get_donuts
  with tracer.start_as_current_span("get_donuts"):
    pp.pprint(trace.get_current_span())
    trace.get_current_span().set_attribute('donut_type', 'sprinkes')
    trace.get_current_span().add_event('donut_order')
    # make an API call, do work
    return "Donuts ordered!"

####  Sending Telemetry to Lightstep (or anywhere else)

There's a number of different options developers can use to recieve data from OpenTelemetry-instrumented libraries -- including just printing it to the console. 

Below, we use [Lightstep's OpenTelemetry launcher](https://github.com/lightstep/otel-launcher-python) to send data to Lightstep.

In [ ]:
! pip install opentelemetry_launcher

# Signup here to get an access token: https://app.lightstep.com/signup/developer
# To find your token: https://docs.lightstep.com/docs/create-and-manage-access-tokens
import getpass
lightstep_access_token=getpass.getpass('Please paste your Lightstep access token: ')
lightstep_project = input('Please paste your Lightstep project name: ')

In [ ]:
from opentelemetry.launcher import configure_opentelemetry

service_name = "tutorial-pyton"

# This automatically configures OpenTelemetry to send data to Lightstep.
# Many other destinations are supported, see: https://pypi.org/search/?q=opentelemetry-exporter
configure_opentelemetry(
    service_name=service_name,
    access_token=lightstep_access_token,
)

print(get_donuts())

print(f'View traces in Lightstep:\nhttps://app.lightstep.com/{lightstep_project}/explorer?filter[0][type]=built-in&filter[0][key]=component&filter[0][value]={service_name}')

#### Next steps

Add OpenTelemetry to your Python library or contribute a plugin! While libraries can directly consume the OpenTelemetry API (as above), many library authors choose to write an OpenTelemetry plugin that is distributed separately from their core library code as part of the OpenTelemetry project.

See the `opentelemetry-python-contrib` project [for lots of plugin examples](https://github.com/open-telemetry/opentelemetry-python-contrib/tree/main/instrumentation).